# C/C++ DLL using MinGW-w64 with Python and Excel VBA


## 1 Using C/C++ DLL with Python

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. But for certain types of calculations Python (and any other interpreted language) can be **very slow**. It is particularly iterations over large arrays that is difficult to do efficiently.

Such calculations may be implemented in a compiled language such as C or Fortran. In Python it is relatively easy to call out to libraries with compiled C or Fortran code. 

But before we go ahead and work on optimizing anything, it is always worthwhile to ask.... 

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

## ctypes

ctypes is a Python library for calling out to C code.

We manually need to load the library and set properties such as the functions return and argument types.

On the otherhand we do not need to touch the C code at all. 

In [1]:
%%file ./code/gcc/funs.h

#ifndef FUNS_H
#define FUNS_H

void hello();
double dprod(double *x, int n);
unsigned long LinearFibonacci(int n);

#endif

Overwriting ./code/gcc/funs.h


In [24]:
%%file ./code/gcc/funs.c

#include <stdio.h>
#include "funs.h"

void hello()
{
   printf("C says Hello world!\n");
}

double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

//The linear time algorithm for finding Fibonacci numbers
unsigned long LinearFibonacci(int n)
{
  unsigned long fib;
  if (n< 0)
      return -1;
  
  // base case 0 or 1
  if (n == 0||n==1)
      return 1;
 
  unsigned long f1=1,f2 = 1;
  for(int i=2; i<=n; i++)
  {
    fib = f1 + f2;
    f2=f1;
    f1=fib;
  }    
  return fib;
}

Overwriting ./code/gcc/funs.c


Compile the C file into a shared library:

In [1]:
!gcc -c -O3 -Wall -fPIC -o funs.o ./code/gcc/funs.c
!gcc -shared -o ./code/gcc/libfuns.dll funs.o

In [6]:
!dir .\code\gcc\lib*.dll

 驱动器 D 中的卷是 cmh
 卷的序列号是 02AB-E07E

 D:\SEUCourse\SE_ThermalEnergy\S2017\home\notebook\code\gcc 的目录

2017/05/11  00:39            48,112 libfibonacci.dll
2017/05/11  02:06            48,363 libfuns.dll
2017/05/10  06:33            48,120 libmathfuns.dll
               3 个文件        144,595 字节
               0 个目录 194,276,503,552 可用字节


In [2]:
%%file makefile

CC=gcc
CFLAGS=-O3 -Wall -fPIC -o 

all: libfuns.dll

libfuns.dll: funsobj
	 $(CC) -o ./code/gcc/libfuns.dll -shared funs.o
	 del funs.o
    
funsobj: ./code/gcc/funs.c
	 $(CC) -c $(CFLAGS) funs.o ./code/gcc/funs.c
     
clean:
	 del .\code\gcc\libfuns.dll

Overwriting makefile


In [3]:
!make

gcc -c -O3 -Wall -fPIC -o  funs.o ./code/gcc/funs.c
gcc -o ./code/gcc/libfuns.dll -shared funs.o
del funs.o


The result is a compiled shared library `libfuns.dll`

In [8]:
!make clean

del .\code\gcc\libfuns.dll


Now we need to write wrapper functions to access the C library: 

* To load the library we use the **ctypes** package, which included in the Python standard library (with extensions from numpy for passing arrays to C). 

* Then we manually set the types of the argument and return values. 

   * cdll.LoadLibrary

   * .argtypes

   * .restype

In [4]:
%%file ./code/gcc/funs.py

from ctypes import c_int,c_long,c_double,c_void_p,cdll
import numpy as np

_lib = cdll.LoadLibrary('./code/gcc/libfuns.dll')
#_lib = np.ctypeslib.load_library('libfuns', '.')

_lib.hello.argtypes = []
_lib.hello.restype  =  c_void_p

_lib.dprod.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), c_int]
_lib.dprod.restype  = c_double

_lib.LinearFibonacci.argtypes = [c_int]
_lib.LinearFibonacci.restype= c_long


def hello():
    return _lib.hello()

def dprod(x):
    n = len(x)
    x = np.asarray(x, dtype=np.float)
    return _lib.dprod(x, int(n))

def LinearFibonacci(n):
    return _lib.LinearFibonacci(int(n))

Overwriting ./code/gcc/funs.py


In [11]:
%%file ./code/gcc/run_hello_c.py

import funs
funs.hello()

Overwriting ./code/gcc/run_hello_c.py


In [12]:
!python .\code\gcc\run_hello_c.py

C says Hello world!


### Product function:

In [8]:
import sys
sys.path.append('./code/gcc')
import funs
funs.dprod([12,3,4,5,6]) 

4320.0

### The linear time algorithm for finding Fibonacci numbers

In [5]:
import sys
sys.path.append('./code/gcc')
import funs
funs.LinearFibonacci(20) 

10946

In [6]:
%timeit funs.LinearFibonacci(20) 

1.84 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## 2 C/C++ DLL using MinGW-w64 for Visual Basic Application with Excel

  For Visual Basic applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an exported function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

#### Step 1: Create your DLL.

VBA can only call `__stdcall` functions, not `__cdecl` functions. So we must export all our functions as `__stdcall`. 

Create a DLL with the following code:

* mathfuns.h

* mathfuns.c

In [8]:
%%file ./code/gcc/mathfuns.h

#ifdef BUILD_DLL
#define DLLPORT __declspec(dllexport)
#else
#define DLLPORT __declspec(dllimport)
#endif

DLLPORT __stdcall double fadd(double a,double b);
DLLPORT __stdcall double fmul(double a, double b);

Overwriting ./code/gcc/mathfuns.h


When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the export directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [7]:
%%file ./code/gcc/mathfuns.c

#include "mathfuns.h"

DLLPORT __stdcall double fadd(double a,double b){
   return (a+b);
}

DLLPORT __stdcall double  fmul(double a, double b){
   return (a*b);
}

Overwriting ./code/gcc/mathfuns.c


In [9]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/gcc/libmathfuns.dll mathfuns.o -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: Predefine name as a macro, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an undecorated alias for the exported function names that is simply **the name of the function** 

In [10]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/gcc/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=./code/gcc/libmathfuns.def

* -static-libgcc

   This option links the GNU libgcc library statically. 
   

* -output-def=libmathfuns.def

    Name of .def file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
%load ./code/gcc/libmathfuns.def

In [18]:
%%file makefile

CC=gcc
CFLAGS=-DBUILD_DLL -o

all: libmathfuns.dll

libmathfuns.dll: mathfunsobj
	 $(CC) -shared -o ./code/gcc/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
	 del mathfuns.o
    
mathfunsobj: ./code/gcc/mathfuns.c
	 $(CC) -c $(CFLAGS) mathfuns.o ./code/gcc/mathfuns.c
     
clean:
	 del .\code\gcc\libmathfuns.dll

Overwriting makefile


In [ ]:
!make

### Step 2: Call DLL from Excel VBA

* 1) libmathfuns.dll in the default dll path of windows c:\windows\system

* 2) demo-vba.xlsm in ./code/gcc/

* 3)  press "ALT+F11" into VBA

create the module *mathfuns* to library call such as:
```vba
Public Declare PtrSafe Function fadd Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
Public Declare PtrSafe Function fmul Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
```
create the module *vbaapi* to call such as:
```
Public Function sadd(ByVal a As Double, ByVal b As Double) As Double
    sadd = mathfuns.fadd(a, b)
End Function
 
Public Function smul(ByVal a As Double, ByVal b As Double) As Double
    smul = mathfuns.fmul(a, b)
End Function
```

* 4)  call in cells
![demo-vba](./code/gcc/demo-vba.jpg)

### 3 call_stdcall DLL from Python

```python
 windll.LoadLibrary
```

In [5]:
%%file ./code/gcc/mathfuns.py

from ctypes import windll,c_double

flib = windll.LoadLibrary('./code/gcc/libmathfuns.dll')


def fadd(a,b):
    flib.fadd.argtypes = [c_double,c_double]
    flib.fadd.restype  = c_double
    return flib.fadd(a,b)

def fmul(a,b):
    flib.fmul.argtypes = [c_double,c_double]
    flib.fmul.restype  = c_double
    return flib.fmul(a,b)


Overwriting ./code/gcc/mathfuns.py


In [7]:
import sys
sys.path.append('./code/gcc')
from mathfuns import *
a=4.6
b=5
print(a,'+',b,'=',fadd(a,b))
print(a,'*',b,'=',fmul(a,b))

4.6 + 5 = 9.6
4.6 * 5 = 23.0


### 4 Call __stdcall DLL from C 

In [11]:
%%file ./code/gcc/mainmathfuns.c

#include <stdio.h>
#include "mathfuns.h"
 
int main() {
    double a=5.3;
    double b=6.1;
    double radd=fadd(a,b);
    double rmul=fmul(a,b);
    printf("%.3f + %.3f = %.3f \n",a,b,radd);
    printf("%.3f * %.3f = %.3f \n",a,b,rmul);
    return 0;
}

Overwriting ./code/gcc/mainmathfuns.c


In [12]:
!gcc -c -o mainmathfuns.o ./code/gcc/mainmathfuns.c 
!gcc -o  ./code/gcc/mainmathfuns.exe mainmathfuns.o -I./code/gcc/ -L./code/gcc/ -lmathfuns

In [13]:
!.\code\gcc\mainmathfuns

5.300 + 6.100 = 11.400 
5.300 * 6.100 = 32.330 


In [14]:
%%file makefile

all: mainmathfuns.exe

clean:
	del .\code\gcc\mainmathfuns.exe

mainmathfuns.exe: mainmathfuns.o ./code/gcc/mathfuns.h 
	gcc -o ./code/gcc/mainmathfuns mainmathfuns.o -I./code/gcc -L./code/gcc/ -lmathfuns
	del *.o

mainmathfuns.o: ./code/gcc/mainmathfuns.c 
	gcc -c ./code/gcc/mainmathfuns.c 

Overwriting makefile


In [15]:
!make

gcc -o ./code/gcc/mainmathfuns mainmathfuns.o -I./code/gcc -L./code/gcc/ -lmathfuns
del *.o


In [16]:
!.\code\gcc\mainmathfuns

5.300 + 6.100 = 11.400 
5.300 * 6.100 = 32.330 


In [17]:
!make clean

del .\code\gcc\mainmathfuns.exe


### Further reading

* The linear time algorithm for finding Fibonacci numbers http://www.catonmat.net/blog/on-the-linear-time-algorithm-for-finding-fibonacci-numbers/


* http://docs.python.org/3/library/ctypes.html
* http://scipy-cookbook.readthedocs.io/items/Ctypes.html
* https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html


* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


* Getting Started with VBA in Office 2010: https://msdn.microsoft.com/library/office/ee814735(v=office.14)
   
* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html


